In [1]:
import torch
import sys
import gc
sys.path.append('..')
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, WrapperConfig, SMOL_LM_135M

/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cfg = WrapperConfig(
    model_id_or_path=SMOL_LM_135M,
    max_samples=1000,
    n_epochs=10,
    data_module_choice="evol_codealpaca_dpo",
    notebook_mode=True,
    tuning_mode="sft_lora",
    eval_data_mode="fixed"
)

wrapper = TrainerWrapper(cfg)
wrapper.init_model()

2024-12-28 01:31:01.460 | INFO     | trl_wrapper.trainer_wrapper:init_model:136 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct


In [ ]:
gc.collect()
torch.cuda.empty_cache()
! echo $CUDA_VISIBLE_DEVICES
! nvidia-smi

In [18]:
wrapper.init_data_module()

2024-12-28 01:44:00.645 | INFO     | model.utils:__init__:104 - Cache dir: ../dataset_caches/evol_code_alpaca_data_module
2024-12-28 01:44:00.688 | INFO     | model.utils:setup:118 - Loading dataset for stage fit
2024-12-28 01:44:00.690 | INFO     | model.utils:setup:120 - Processing dataset for stage fit, workers: 1, cache dir ../dataset_caches/evol_code_alpaca_data_module
2024-12-28 01:44:00.691 | INFO     | dataset.code:load_dataset:110 - Loading dataset


evol_codealpaca_dpo


Map:  13%|█▎        | 5000/39483 [00:00<00:00, 41795.59 examples/s]

dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])


Map:  43%|████▎     | 17000/39483 [00:00<00:00, 47282.66 examples/s]

...

dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])


Map: 100%|██████████| 39483/39483 [00:00<00:00, 45460.97 examples/s]

dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])
dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])


Map: 100%|██████████| 399/399 [00:00<00:00, 20906.55 examples/s]
2024-12-28 01:44:02.789 | WARNING  | model.utils:setup:165 - Columns in dataset: ['system', 'question', 'chosen', 'rejected', 'conversations'] do not match torch columns, not setting format


dict_keys(['system', 'question', 'chosen', 'rejected'])
dict_keys(['conversations'])


In [19]:
wrapper.data_module.train_dataset['conversations'][0]

[{'content': 'You are experienced in guiding both individual developers and teams, offering mentorship in personal growth, coding skills improvement, and collaborative development.',
  'role': 'system'},
 {'content': "The similarity measure between two attributes always falls in the range of $[0,1]$, why is this not true in the case of dissimilarity where the value falls in the range of $[0, \\infty)$?\nCan't the dissimilarity between them be thought of as the opposite of similarity in the sense that if they have a similarity of 0, then the dissimilarity will be 1 and vice versa?",
  'role': 'user'}]

In [13]:
wrapper.init_trainer()

2024-12-28 01:38:08.595 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:205 - LoRA already loaded, ignoring
2024-12-28 01:38:08.602 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:214 - Initializing trainer, run: run-12-28-1-38-96911, project: codecontests-llama-3b
2024-12-28 01:38:08.603 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:217 - logprobs cache location: dataset_caches/evol_code_alpaca_data_module/b68b303b/ref_logprobs_cache peft config: True
2024-12-28 01:38:08.603 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:220 - WrapperConfig(model_id_or_path='HuggingFaceTB/SmolLM2-135M-Instruct', notebook_mode=True, max_sequence_length=1512, max_prompt_length=1024, max_eval_sample_length=1024, max_samples=1000, train_batch_size=4, eval_batch_size=2, gradient_accumulation_steps=1, gradient_checkpointing=True, root_dir=None, data_module_choice='evol_codealpaca_dpo', wandb_project_name='codecontests-llama-3b', n_epochs=10, max_eval_dataset_size=None, adapter_pat

/admin/home-brianf/smolmodels/notebooks/../trl_wrapper/trainer_wrapper.py:254: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  self.trainer = SFTTrainer(
Map:   0%|          | 0/39483 [00:00<?, ? examples/s]


ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
wrapper.train()

In [ ]:
from IPython.display import Markdown
for prompt, chosen, rejected, logprob_diff in []:
    display(Markdown(f"\n### Prompt: {prompt}\n\n### Chosen:\n {chosen}\n\n### Rejected:\n {rejected}\n\nLogprob diff: {logprob_diff}"))